In [14]:
import torch
import numpy as np
import torchvision
from torchinfo import summary

from ultralytics import YOLO

We will use the YOLO12s model from ultralytics (2.61 ms on T4, 9.3M params)

Object classes:
- 'Car', 'Van', 'Truck',
- 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram',
- 'Misc' or 'DontCare'

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = YOLO("yolo12s.pt")
model.model.to(device)

# NCHW
summary(model.model, input_size=(1, 3, 640, 640), device=device)


Layer (type:depth-idx)                                       Output Shape              Param #
DetectionModel                                               [1, 84, 8400]             --
├─Sequential: 1-1                                            --                        --
│    └─Conv: 2-1                                             [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-1                                      [1, 32, 320, 320]         (864)
│    │    └─BatchNorm2d: 3-2                                 [1, 32, 320, 320]         (64)
│    └─Detect: 2-132                                         --                        (recursive)
│    │    └─ModuleList: 3-153                                --                        (recursive)
│    └─Conv: 2-3                                             [1, 64, 160, 160]         --
│    │    └─Conv2d: 3-4                                      [1, 64, 160, 160]         (18,432)
│    │    └─BatchNorm2d: 3-5                                 [1, 6